In [17]:
GROQ_API_KEY = "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"
NGROK_AUTH_TOKEN = "314kmzWnhzw3GWwueTDm189SluI_7u4KaGdyxRhDUYx8s9ZN3"

import os
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
# !pip install sentence-transformers langchain-community faiss-cpu tqdm pyngrok streamlit

from google.colab import files
from tqdm import tqdm
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

print("📥 Downloading embedding model...")
_ = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 📂 Upload PDF
uploaded = files.upload()
PDF_FILE_PATH = list(uploaded.keys())[0]

# Load PDF
def load_single_pdf(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load()

# Split into chunks
def create_chunks(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100, add_start_index=True)
    return splitter.split_documents(documents)

print("⚡ Loading PDF...")
documents = load_single_pdf(PDF_FILE_PATH)
print(f"✅ Loaded {len(documents)} documents.")

print("✂ Splitting document into chunks...")
chunks = create_chunks(documents)
print(f"✅ Created {len(chunks)} chunks.")

print("🔹 Creating FAISS DB...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_db = FAISS.from_documents(tqdm(chunks, desc="Generating embeddings"), embeddings)
os.makedirs("vectorstore", exist_ok=True)
faiss_db.save_local("vectorstore/database.faiss")
print("✅ FAISS DB saved!")

# Streamlit app file
app_code = '''
import streamlit as st
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

FAISS_DATABASE_PATH = "vectorstore/database.faiss"

def get_embedding():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def load_faiss():
    if os.path.exists(FAISS_DATABASE_PATH):
        return FAISS.load_local(
            FAISS_DATABASE_PATH,
            get_embedding(),
            allow_dangerous_deserialization=True
        )
    else:
        raise FileNotFoundError("FAISS DB not found")

faiss_db = load_faiss()
API_KEY = os.getenv("GROQ_API_KEY") or "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"
llm_model = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=API_KEY)

def retrieve_docs(query, k=5):
    return faiss_db.similarity_search(query, k=k)

def get_context(documents):
    return "\\n\\n".join([doc.page_content for doc in documents])

custom_prompt_template = """USE THE PIECES OF INFORMATION PROVIDED IN CONTEXT TO ANSWER USER'S QUESTION.
IF YOU DON'T KNOW THE ANSWER, JUST SAY "I DON'T KNOW".
QUESTION: {QUESTION}
CONTEXT: {CONTEXT}
ANSWER:
"""

def generate_answer(documents, model, query):
    context = get_context(documents)
    prompt = ChatPromptTemplate.from_template(custom_prompt_template)
    chain = prompt | model
    return chain.invoke({"QUESTION": query, "CONTEXT": context})

st.title("AERODYNAMICS Chat")

user_query = st.text_area(
    "Please provide your query here",
    height=100,
    placeholder="Ask a question about the uploaded file or general questions..."
)

if st.button("Ask Question"):
    if not user_query.strip():
        st.error("Please enter a query.")
    else:
        st.chat_message("user").write(user_query)
        retrieved_docs = retrieve_docs(user_query)
        response = generate_answer(retrieved_docs, model=llm_model, query=user_query)
        st.chat_message("assistant").write(response)
'''

with open("CHAT_APP.py", "w") as f:
    f.write(app_code)

#  Streamlit with ngrok
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "314kmzWnhzw3GWwueTDm189SluI_7u4KaGdyxRhDUYx8s9ZN3"  
!ngrok authtoken {NGROK_AUTH_TOKEN}

get_ipython().system_raw('streamlit run CHAT_APP.py --server.port 8501 &')

public_url = ngrok.connect(8501)
print("🚀 Your chatbot is live at:", public_url)


📥 Downloading embedding model...


Saving Fundamentals-of-aerodynamics-6-Edition.pdf to Fundamentals-of-aerodynamics-6-Edition (7).pdf
⚡ Loading PDF...
✅ Loaded 1154 documents.
✂ Splitting document into chunks...
✅ Created 2160 chunks.
🔹 Creating FAISS DB...


Generating embeddings: 100%|██████████| 2160/2160 [00:00<00:00, 695348.58it/s]


✅ FAISS DB saved!
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Your chatbot is live at: NgrokTunnel: "https://70e33babc5dd.ngrok-free.app" -> "http://localhost:8501"
